In [36]:
import numpy as np
import pandas as pd

from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from xgboost import XGBClassifier
import xgboost

import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("loan-default-prediction")

<Experiment: artifact_location='/workspaces/loan-bot/mlruns/1', creation_time=1731172818718, experiment_id='1', last_update_time=1731172818718, lifecycle_stage='active', name='loan-default-prediction', tags={}>

In [12]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [34]:
train_df = pd.read_csv("./data/clean_train_data.csv")
eval_df = pd.read_csv("./data/clean_test_data.csv")

In [37]:
X_train, y_train = train_df.drop(columns=['loan_status']), train_df.loan_status
X_test, y_test = eval_df.drop(columns=['loan_status']), eval_df.loan_status

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

X_train = np.array(X_train).astype(np.float32)
X_test = np.array(X_test).astype(np.float32)


# y_train = np.array(y_train).astype(int)
# y_test = np.array(y_test).astype(int)

In [41]:
train = xgboost.DMatrix(X_train, label=y_train)
valid = xgboost.DMatrix(X_test, label=y_test)

In [ ]:
def objective(space):
    """
    Objective function for hyperopt optimization with additional error checking
    """
    # Log current iteration parameters
    logger.info(f"Trying parameters: {space}")
    
    try:
        # Define base parameters that aren't being tuned
        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'tree_method': 'hist',
            'max_depth': int(space['max_depth']),
            'min_child_weight': space['min_child_weight'],
            'learning_rate': space['learning_rate'],
            'subsample': space['subsample'],
            'colsample_bytree': space['colsample_bytree'],
            'gamma': space['gamma'],
            'verbosity': 0  # Reduce XGBoost verbosity
        }
        
        # Verify DMatrix labels before training
        unique_labels = np.unique(dtrain.get_label())
        logger.info(f"Labels in DMatrix before CV: {unique_labels}")
        
        # Create cross-validation folds with error catching
        cv_result = xgboost.cv(
            params,
            dtrain,
            num_boost_round=1000,
            nfold=5,
            early_stopping_rounds=50,
            verbose_eval=False,
            seed=42  # Add seed for reproducibility
        )
        
        # Get the best score
        best_score = cv_result['test-logloss-mean'].min()
        logger.info(f"Iteration complete. Best score: {best_score}")
        
        return {'loss': best_score, 'status': STATUS_OK}
    
    except Exception as e:
        logger.error(f"Error in objective function: {str(e)}")
        logger.error(f"Parameters that caused error: {space}")
        # Return a high loss value instead of failing
        return {'loss': float('inf'), 'status': STATUS_OK}

In [63]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('developer', 'jgd')
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
         
        
        unique_labels = np.unique(train.get_label())
        print(f"Labels in DMatrix : {unique_labels}")
        
        booster = xgboost.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals = [(valid, 'validation')],
            early_stopping_rounds=50,
        )
        
        predt = booster.predict(valid)
        y_pred = (predt > 0.5).astype(int)
        true_labels = valid.get_label()
        
        try:
            f1 = f1_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)

            mlflow.log_metric("f1", f1)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("auc", auc)
        except:
            f1=np.NaN
            print("error in training")
        
        return {"loss": -f1, 'status': STATUS_OK}
    

        
        
        

In [64]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'binary:logistic',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

Labels in DMatrix : [0. 1.]                                                                                                              
[0]	validation-logloss:0.33505                                                                                                           
[1]	validation-logloss:0.30321                                                                                                           
[2]	validation-logloss:0.28862                                                                                                           
[3]	validation-logloss:0.28379                                                                                                           
[4]	validation-logloss:0.28218                                                                                                           
[5]	validation-logloss:0.28322                                                                                                           
[6]	validation-logloss:0.28393    

[3]	validation-logloss:0.39553                                                                                                           
[4]	validation-logloss:0.40317                                                                                                           
[5]	validation-logloss:0.41021                                                                                                           
[6]	validation-logloss:0.41736                                                                                                           
[7]	validation-logloss:0.42356                                                                                                           
[8]	validation-logloss:0.42947                                                                                                           
[9]	validation-logloss:0.43438                                                                                                           
[10]	validation-logloss:0.43946   

[9]	validation-logloss:0.27359                                                                                                           
[10]	validation-logloss:0.27306                                                                                                          
[11]	validation-logloss:0.27298                                                                                                          
[12]	validation-logloss:0.27235                                                                                                          
[13]	validation-logloss:0.27336                                                                                                          
[14]	validation-logloss:0.27317                                                                                                          
[15]	validation-logloss:0.27327                                                                                                          
[16]	validation-logloss:0.27305   

[4]	validation-logloss:0.37973                                                                                                           
[5]	validation-logloss:0.36697                                                                                                           
[6]	validation-logloss:0.35591                                                                                                           
[7]	validation-logloss:0.34630                                                                                                           
[8]	validation-logloss:0.33793                                                                                                           
[9]	validation-logloss:0.33028                                                                                                           
[10]	validation-logloss:0.32367                                                                                                          
[11]	validation-logloss:0.31790   

[63]	validation-logloss:0.27406                                                                                                          
[64]	validation-logloss:0.27429                                                                                                          
[65]	validation-logloss:0.27445                                                                                                          
[66]	validation-logloss:0.27462                                                                                                          
[67]	validation-logloss:0.27486                                                                                                          
[68]	validation-logloss:0.27511                                                                                                          
[69]	validation-logloss:0.27532                                                                                                          
[70]	validation-logloss:0.27551   

...

[65]	validation-logloss:0.30740                                                                                                          
[66]	validation-logloss:0.30788                                                                                                          
[67]	validation-logloss:0.30843                                                                                                          
[68]	validation-logloss:0.30895                                                                                                          
[69]	validation-logloss:0.30948                                                                                                          
[70]	validation-logloss:0.31001                                                                                                          
[71]	validation-logloss:0.31055                                                                                                          
[72]	validation-logloss:0.31099   

[41]	validation-logloss:0.27399                                                                                                          
[42]	validation-logloss:0.27312                                                                                                          
[43]	validation-logloss:0.27251                                                                                                          
[44]	validation-logloss:0.27181                                                                                                          
[45]	validation-logloss:0.27130                                                                                                          
[46]	validation-logloss:0.27073                                                                                                          
[47]	validation-logloss:0.27010                                                                                                          
[48]	validation-logloss:0.26957   

[100]	validation-logloss:0.26319                                                                                                         
[101]	validation-logloss:0.26320                                                                                                         
[102]	validation-logloss:0.26325                                                                                                         
[103]	validation-logloss:0.26323                                                                                                         
[104]	validation-logloss:0.26327                                                                                                         
[105]	validation-logloss:0.26331                                                                                                         
[106]	validation-logloss:0.26335                                                                                                         
[107]	validation-logloss:0.26340  

[19]	validation-logloss:0.29537                                                                                                          
[20]	validation-logloss:0.29273                                                                                                          
[21]	validation-logloss:0.29042                                                                                                          
[22]	validation-logloss:0.28819                                                                                                          
[23]	validation-logloss:0.28620                                                                                                          
[24]	validation-logloss:0.28431                                                                                                          
[25]	validation-logloss:0.28271                                                                                                          
[26]	validation-logloss:0.28115   

[78]	validation-logloss:0.26845                                                                                                          
[79]	validation-logloss:0.26850                                                                                                          
[80]	validation-logloss:0.26869                                                                                                          
[81]	validation-logloss:0.26880                                                                                                          
[82]	validation-logloss:0.26894                                                                                                          
[83]	validation-logloss:0.26902                                                                                                          
[84]	validation-logloss:0.26919                                                                                                          
[85]	validation-logloss:0.26932   